# regardless of affiliation

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
import os
import itertools
import warnings
import glob
import sys
staff = pd.read_csv("people_flt.csv")
staff_flt = staff[(~staff["MTMT"].isnull()) & (staff["MTMT"]!=11111111) & (staff["MTMT"]!=0) & (staff["Munkakör"].str.lower()!="doktorandusz") & (staff["Munkakör"].str.lower()!="sh doktorandusz")]
staff_flt["MTMT"] = staff_flt["MTMT"].astype("int")
def load_pub_for_author(mtid):
    page = 1
    size = 1000
    tries = 1
    author_pubs = []
    while True:
        os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?cond=published;eq;true&cond=core;eq;true&cond=authors.mtid;eq;{mtid}&sort=publishedYear,desc&sort=firstAuthor,asc&size={size}&fields=citations,pubStats&labelLang=hun&page={page}&format=json" >wget.out 2>wget.err')
        with open("tmp.json", "r") as f:
            try:
                dta = json.load(f)
                if page == 1 and len(dta["content"]) == 0:
                    return []
                author_pubs += dta["content"]
                if not dta["paging"]["last"]:
                    page += 1
                else:
                    break
            except:
                tries += 1
                print("Error, retrying")
                if tries > 30:
                    return []
    return author_pubs
for mtid in tqdm(staff_flt["MTMT"]):
    if not os.path.exists(f"3_1/{mtid}.json"):
        json.dump(load_pub_for_author(mtid), open(f"3_1/{mtid}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)

/tmp/ipykernel_271585/3833529470.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["MTMT"] = staff_flt["MTMT"].astype("int")


In [2]:
def load_pub_for_author(mtid):
    page = 1
    size = 1000
    tries = 1
    author_pubs = []
    while True:
        os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?cond=published;eq;true&cond=core;eq;true&cond=authors.mtid;eq;{mtid}&sort=publishedYear,desc&sort=firstAuthor,asc&size={size}&fields=citations,pubStats&labelLang=hun&page={page}&format=json" >wget.out 2>wget.err')
        with open("tmp.json", "r") as f:
            try:
                dta = json.load(f)
                if page == 1 and len(dta["content"]) == 0:
                    return []
                author_pubs += dta["content"]
                if not dta["paging"]["last"]:
                    page += 1
                else:
                    break
            except:
                tries += 1
                print("Error, retrying")
                if tries > 30:
                    return []
    return author_pubs

In [3]:
for mtid in tqdm(staff_flt["MTMT"]):
    if not os.path.exists(f"3_1/{mtid}.json"):
        json.dump(load_pub_for_author(mtid), open(f"3_1/{mtid}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)

100%|█████████████████████████████████████| 119/119 [00:00<00:00, 118725.54it/s]


In [4]:
ifdf = pd.read_csv('./pubgraph/ifdf_v8_2021.csv')
ifdf.loc[ifdf["if"].isnull(), "if"] = 0.0
maxIFYear = ifdf.year.max()
ifdf_e = ifdf.copy().set_index(["year", "eissn"]).sort_index()
ifdf_p = ifdf.copy().set_index(["year", "pissn"]).sort_index()

def getif(pub):
    if "journal" not in pub:
        return 0.0, 1.0
    year = min(pub["publishedYear"], maxIFYear)
    if "pIssn" in pub["journal"]:
        issn = pub["journal"]["pIssn"]
        if (year,issn) in ifdf_p.index:
            ifval = ifdf_p.loc[(year,issn),"if"]
            catif = ifdf_p.loc[(year,issn),"categoryMedianIf"]
            if len(ifval) == 1 and float(ifval) > 0.0:
                if len(catif) == 1 and float(catif) > 0.0:
                    return float(ifval), float(catif)
                else:
                    return float(ifval), float(ifval) # ha if-es az újság, de még nem elég régóta. nincs sok ilyen.
    if "eIssn" in pub["journal"]:
        issn = pub["journal"]["eIssn"]
        if (year,issn) in ifdf_e.index:
            ifval = ifdf_e.loc[(year,issn),"if"]
            catif = ifdf_e.loc[(year,issn),"categoryMedianIf"]
            if len(ifval) == 1 and float(ifval) > 0.0:
                if len(catif) == 1 and float(catif) > 0.0:
                    return float(ifval), float(catif)
                else:
                    return float(ifval), float(ifval) # ha if-es az újság, de még nem elég régóta. nincs sok ilyen.
    return 0.0, 1.0

# a függvény visszaadja a folyóirat rangját (D1, Q1, Q2, ...)
def getrating(pub):
    if "ratings" not in pub:
        return ""
    for r in pub["ratings"]:
        if r["otype"] == "SjrRating" and "ranking" in r:
            return r["ranking"]
    return ""


In [5]:
def score_person(mtid):
    if not os.path.exists(f'3_1/{mtid}.json'):
        raise Exception("Nincs letöltve a publikációs lista!")
    pubs = json.load(open(f'3_1/{mtid}.json'))
    score = score_publist([mtid], pubs)
    score["mtid"] = int(mtid)
    return score
    
def score_publist(mtid_list, pubs):
    score = {"pub_count": 0, "q_paper": 0, "q_n_paper": 0, "q_book": 0, "if": 0, "relif": 0, "ifcnt": 0, "if_norm": 0, "relif_norm": 0, "h": 0, "i": 0, "first_pub": 2100, "last_pub": 0}
    cit_list = []
    useShare = False
    ix = 0
    for pub in pubs:
        ix += 1
        if "error" in pub and pub["error"]!="VALIDATION_ERROR":
            continue
        if "category" not in pub or pub["category"]["label"]!="Tudományos":
            continue
        # compute share of the authos
        authors = 0
        share = len(mtid_list)/len(pub["authorships"]) if len(pub["authorships"])>0 else 0
        for a in pub["authorships"]:
            if "author" in a and a["author"]["mtid"] in mtid_list and a["authorTyped"]:
                authors += 1
        if authors < len(mtid_list):
            continue
        # compute number of pages
        plength = 0
        try:
            if "pageLength" in pub:
                plength = int(pub["pageLength"])
            elif "firstPage" in pub and "lastPage" in pub:
                plength = int(pub["lastPage"]) - int(pub["firstPage"]) + 1            
        except:
            pass
        # compute impact factor
        ifct, nrm = getif(pub)        
        # compute Q score
        if pub["otype"] == "JournalArticle" and "journal" in pub:
            if "fullPublication" in pub and pub["fullPublication"] and "reviewType" in pub["journal"] and pub["journal"]["reviewType"]=="REVIEWED" and "subType" in pub and \
                pub["subType"]["name"] in ["Szakcikk", "Összefoglaló cikk", "Konferenciaközlemény", "Rövid közlemény", "Sokszerzős vagy csoportos szerzőségű szakcikk"]:                
                if ifct > 0:
                    totalscr = max(0.6,ifct)
                elif "foreignEdition" in pub and pub["foreignEdition"]:
                    totalscr = 0.4
                else:
                    totalscr = 0.3
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share / nrm if nrm > 0 else totalscr * share
                if ifct > 0:
                    score["if"] += ifct
                    score["relif"] += totalscr * share
                    score["ifcnt"] += 1
                    score["if_norm"] += ifct / nrm if nrm > 0 else ifct
                    score["relif_norm"] += totalscr * share / nrm if nrm > 0 else totalscr * share
        elif "fullPublication" in pub and pub["fullPublication"] and \
            ((pub["type"]["label"]=="Könyvrészlet" and "subType" in pub and pub["subType"]["label"]=="Konferenciaközlemény (Könyvrészlet)") or pub["type"]["label"]=="Egyéb konferenciaközlemény"):                
            if plength >= 4:
                totalscr = 0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share
        elif pub["type"]["label"] == "Könyv" or \
            (pub["type"]["label"] == "Könyvrészlet" and "subType" in pub and pub["subType"]["label"] in ["Könyvfejezet (Könyvrészlet)", "Szaktanulmány (Könyvrészlet)"]):
            if plength >= 10:
                if plength >= 100:
                    totalscr = 2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 1
                else:
                    totalscr = (0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1) * math.floor(plength/10)
                score["q_book"] += totalscr * share
        # Process citations
        if "independentCitationCount" in pub:
            score["i"] += pub["independentCitationCount"]
            cit_list.append(pub["independentCitationCount"])
        if "publishedYear" in pub:
            score["last_pub"] = max(score["last_pub"], pub["publishedYear"])
            score["first_pub"] = min(score["first_pub"], pub["publishedYear"])
        score["pub_count"] += 1
    score["q"] = score["q_paper"] + np.minimum(score["q_book"], 3.0)
    score["qn"] = score["q_n_paper"] + np.minimum(score["q_book"], 3.0)
    # h-index
    cit_list.sort(reverse=True)
    while score["h"] < len(cit_list) and score["h"] < cit_list[score["h"]]:
        score["h"] += 1
        
    return score

In [6]:
def get_h_score(mtid):
    cit_list = []
    if os.path.exists(f'3_1/{mtid}.json'):
        with open(f'3_1/{mtid}.json') as f:
            try:
                js = json.load(f)
                for pub in js:
                    if "independentCitationCount" in pub:
                        cit_list.append(pub["independentCitationCount"])
            except:
                pass
    cit_list.sort(reverse=True)
    hindex = 0
    while hindex < len(cit_list) and hindex < cit_list[hindex]:
        hindex += 1
    return hindex

In [7]:
staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]

/tmp/ipykernel_271585/2996616290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]


In [8]:
dta = {"MTMT": [], "pubCount": [], "qScore": [], "qnScore": [], "ifCount": [], "ifScore": [], "relifScore": [], "ifNormScore": [], "relifNormScore": [], "citations": [], "hIndex": [], "firstPub": [], "lastPub": []}
for idx, prs in staff_flt.iterrows():
    scr = score_person(prs["MTMT"])
    dta["MTMT"].append(scr["mtid"])
    dta["pubCount"].append(scr["pub_count"])
    dta["qScore"].append(scr["q"])
    dta["qnScore"].append(scr["qn"])
    dta["ifCount"].append(scr["ifcnt"])
    dta["ifScore"].append(scr["if"])
    dta["relifScore"].append(scr["relif"])
    dta["ifNormScore"].append(scr["if_norm"])
    dta["relifNormScore"].append(scr["relif_norm"])
    dta["citations"].append(scr["i"])
    dta["hIndex"].append(scr["h"])
    dta["firstPub"].append(scr["first_pub"])
    dta["lastPub"].append(scr["last_pub"])
node_person = pd.DataFrame(dta)
pd.set_option('display.max_rows', 15)
node_person

,MTMT,pubCount,qScore,qnScore,ifCount,ifScore,relifScore,ifNormScore,relifNormScore,citations,hIndex,firstPub,lastPub
0,10012522,23,9.162250,6.800885,10,10.856,8.162250,7.796477,5.800885,98,7,1998,2020
1,10041405,36,7.200000,16.093861,6,0.918,2.400000,4.281639,11.293861,42,3,1970,2009
2,10011729,33,15.860783,12.435711,26,39.425,15.727450,28.957170,12.302377,219,8,1997,2021
3,10042381,15,1.733333,1.733333,0,0.000,0.000000,0.000000,0.000000,3,1,2008,2017
4,10011669,31,7.139625,3.533433,10,21.269,5.916898,8.404271,2.310706,168,8,1985,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,10019824,20,8.937917,7.132186,15,15.574,8.287917,12.140872,6.482186,200,6,2006,2022
115,10019824,20,8.937917,7.132186,15,15.574,8.287917,12.140872,6.482186,200,6,2006,2022
116,10042219,39,29.287500,14.404563,30,67.256,29.220833,31.425362,14.337897,144,7,2008,2023
117,10015273,33,25.605867,15.803946,30,53.344,25.372533,33.254772,15.570612,328,10,2001,2022


In [9]:
people=pd.read_csv('people_flt.csv')
scores=node_person
scores=pd.merge(scores, people[["MTMT", "Név", "Web"]], how='inner',  on=["MTMT"])
list1=scores.sort_values(by=["ifScore"],ascending=False)
list3=scores.sort_values(by=["citations"],ascending=False)
list4=scores.sort_values(by=["citations"],ascending=False)
list6=scores.sort_values(by=["hIndex"],ascending=False)
list1[[ "Név","ifScore"]].head(10)


,Név,ifScore
63,Molnár Lajos Gábor,100.458
20,Faragó István György,93.602
38,Karátson János,75.252
86,Rónyai Lajos,70.879
110,Tóth János,69.834
118,Vrana Péter,67.256
49,Kovács Mihály,63.502
59,Matolcsi Máté,56.742
119,Weiner Mihály,53.344
18,Etesi Gábor,53.291


In [10]:
list3[[ "Név","citations"]].head(10)

,Név,citations
14,Csiszár Imre,14371
47,Kornai András,3187
63,Molnár Lajos Gábor,1821
86,Rónyai Lajos,1364
59,Matolcsi Máté,914
67,Mosonyi Milán,830
92,Simonovits András,812
20,Faragó István György,811
99,Szántai Tamás,761
100,Szász Domokos,749


In [11]:
list6[[ "Név","hIndex"]].head(10)

,Név,hIndex
14,Csiszár Imre,40
47,Kornai András,26
63,Molnár Lajos Gábor,22
86,Rónyai Lajos,21
100,Szász Domokos,17
67,Mosonyi Milán,17
49,Kovács Mihály,17
91,Simon Károly,16
99,Szántai Tamás,15
59,Matolcsi Máté,14


# last 5 years

In [12]:
def load_pub_for_author(mtid):
    page = 1
    size = 1000
    tries = 1
    author_pubs = []
    while True:
        os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?cond=published;eq;true&cond=core;eq;true&cond=authors.mtid;eq;{mtid}&cond=publishedYear;range;2017%2C2023&sort=publishedYear,desc&sort=firstAuthor,asc&size={size}&fields=citations,pubStats&labelLang=hun&page={page}&format=json" >wget.out 2>wget.err')
        with open("tmp.json", "r") as f:
            try:
                dta = json.load(f)
                if page == 1 and len(dta["content"]) == 0:
                    return []
                author_pubs += dta["content"]
                if not dta["paging"]["last"]:
                    page += 1
                else:
                    break
            except:
                tries += 1
                print("Error, retrying")
                if tries > 30:
                    return []
    return author_pubs
for mtid in tqdm(staff_flt["MTMT"]):
    if not os.path.exists(f"3_2/{mtid}.json"):
        json.dump(load_pub_for_author(mtid), open(f"3_2/{mtid}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)

100%|██████████████████████████████████████| 119/119 [00:00<00:00, 86323.45it/s]


In [40]:
def score_person(mtid):
    if not os.path.exists(f'3_2/{mtid}.json'):
        raise Exception("Nincs letöltve a publikációs lista!")
    pubs = json.load(open(f'3_2/{mtid}.json'))
    score = score_publist([mtid], pubs)
    score["mtid"] = int(mtid)
    return score
    
def score_publist(mtid_list, pubs):
    score = {"pub_count": 0, "q_paper": 0, "q_n_paper": 0, "q_book": 0, "if": 0, "relif": 0, "ifcnt": 0, "if_norm": 0, "relif_norm": 0, "h": 0, "i": 0, "first_pub": 2100, "last_pub": 0}
    cit_list = []
    useShare = False
    ix = 0
    for pub in pubs:
        ix += 1
        if "error" in pub and pub["error"]!="VALIDATION_ERROR":
            continue
        if "category" not in pub or pub["category"]["label"]!="Tudományos":
            continue
        # compute share of the authos
        authors = 0
        share = len(mtid_list)/len(pub["authorships"]) if len(pub["authorships"])>0 else 0
        for a in pub["authorships"]:
            if "author" in a and a["author"]["mtid"] in mtid_list and a["authorTyped"]:
                authors += 1
        if authors < len(mtid_list):
            continue
        # compute number of pages
        plength = 0
        try:
            if "pageLength" in pub:
                plength = int(pub["pageLength"])
            elif "firstPage" in pub and "lastPage" in pub:
                plength = int(pub["lastPage"]) - int(pub["firstPage"]) + 1            
        except:
            pass
        # compute impact factor
        ifct, nrm = getif(pub)        
        # compute Q score
        if pub["otype"] == "JournalArticle" and "journal" in pub:
            if "fullPublication" in pub and pub["fullPublication"] and "reviewType" in pub["journal"] and pub["journal"]["reviewType"]=="REVIEWED" and "subType" in pub and \
                pub["subType"]["name"] in ["Szakcikk", "Összefoglaló cikk", "Konferenciaközlemény", "Rövid közlemény", "Sokszerzős vagy csoportos szerzőségű szakcikk"]:                
                if ifct > 0:
                    totalscr = max(0.6,ifct)
                elif "foreignEdition" in pub and pub["foreignEdition"]:
                    totalscr = 0.4
                else:
                    totalscr = 0.3
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share / nrm if nrm > 0 else totalscr * share
                if ifct > 0:
                    score["if"] += ifct
                    score["relif"] += totalscr * share
                    score["ifcnt"] += 1
                    score["if_norm"] += ifct / nrm if nrm > 0 else ifct
                    score["relif_norm"] += totalscr * share / nrm if nrm > 0 else totalscr * share
        elif "fullPublication" in pub and pub["fullPublication"] and \
            ((pub["type"]["label"]=="Könyvrészlet" and "subType" in pub and pub["subType"]["label"]=="Konferenciaközlemény (Könyvrészlet)") or pub["type"]["label"]=="Egyéb konferenciaközlemény"):                
            if plength >= 4:
                totalscr = 0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share
        elif pub["type"]["label"] == "Könyv" or \
            (pub["type"]["label"] == "Könyvrészlet" and "subType" in pub and pub["subType"]["label"] in ["Könyvfejezet (Könyvrészlet)", "Szaktanulmány (Könyvrészlet)"]):
            if plength >= 10:
                if plength >= 100:
                    totalscr = 2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 1
                else:
                    totalscr = (0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1) * math.floor(plength/10)
                score["q_book"] += totalscr * share
        # Process citations
        if "independentCitationCount" in pub:
            score["i"] += pub["independentCitationCount"]
            cit_list.append(pub["independentCitationCount"])
        if "publishedYear" in pub:
            score["last_pub"] = max(score["last_pub"], pub["publishedYear"])
            score["first_pub"] = min(score["first_pub"], pub["publishedYear"])
        score["pub_count"] += 1
    score["q"] = score["q_paper"] + np.minimum(score["q_book"], 3.0)
    score["qn"] = score["q_n_paper"] + np.minimum(score["q_book"], 3.0)
    # h-index
    cit_list.sort(reverse=True)
    while score["h"] < len(cit_list) and score["h"] < cit_list[score["h"]]:
        score["h"] += 1
        
    return score
def get_h_score(mtid):
    cit_list = []
    if os.path.exists(f'3_2/{mtid}.json'):
        with open(f'3_2/{mtid}.json') as f:
            try:
                js = json.load(f)
                for pub in js:
                    if "independentCitationCount" in pub:
                        cit_list.append(pub["independentCitationCount"])
            except:
                pass
    cit_list.sort(reverse=True)
    hindex = 0
    while hindex < len(cit_list) and hindex < cit_list[hindex]:
        hindex += 1
    return hindex

In [41]:
staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]
dta = {"MTMT": [], "pubCount": [], "qScore": [], "qnScore": [], "ifCount": [], "ifScore": [], "relifScore": [], "ifNormScore": [], "relifNormScore": [], "citations": [], "hIndex": [], "firstPub": [], "lastPub": []}
for idx, prs in staff_flt.iterrows():
    scr = score_person(prs["MTMT"])
    dta["MTMT"].append(scr["mtid"])
    dta["pubCount"].append(scr["pub_count"])
    dta["qScore"].append(scr["q"])
    dta["qnScore"].append(scr["qn"])
    dta["ifCount"].append(scr["ifcnt"])
    dta["ifScore"].append(scr["if"])
    dta["relifScore"].append(scr["relif"])
    dta["ifNormScore"].append(scr["if_norm"])
    dta["relifNormScore"].append(scr["relif_norm"])
    dta["citations"].append(scr["i"])
    dta["hIndex"].append(scr["h"])
    dta["firstPub"].append(scr["first_pub"])
    dta["lastPub"].append(scr["last_pub"])
node_person = pd.DataFrame(dta)
node_person

/tmp/ipykernel_271585/3273158239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]


,MTMT,pubCount,qScore,qnScore,ifCount,ifScore,relifScore,ifNormScore,relifNormScore,citations,hIndex,firstPub,lastPub
0,10012522,3,1.527500,0.762877,2,3.055,1.527500,1.525753,0.762877,17,2,2017,2020
1,10041405,0,0.000000,0.000000,0,0.000,0.000000,0.000000,0.000000,0,0,2100,0
2,10011729,8,3.312200,1.720991,6,10.801,3.178867,5.417736,1.587658,25,3,2017,2021
3,10042381,2,0.200000,0.200000,0,0.000,0.000000,0.000000,0.000000,0,0,2017,2017
4,10011669,5,1.935000,0.879538,1,6.140,1.535000,1.918151,0.479538,8,1,2017,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,10019824,7,4.052500,2.776344,6,6.675,4.052500,4.616211,2.776344,11,3,2017,2022
115,10019824,7,4.052500,2.776344,6,6.675,4.052500,4.616211,2.776344,11,3,2017,2022
116,10042219,24,17.711183,8.405214,18,39.871,17.644517,18.081074,8.338547,30,3,2017,2023
117,10015273,13,10.910750,5.790201,13,24.309,10.910750,13.781714,5.790201,85,5,2017,2022


In [42]:
people=pd.read_csv('people_flt.csv')
scores=node_person
scores=pd.merge(scores, people[["MTMT", "Név", "Web"]], how='inner',  on=["MTMT"])
list1=scores.sort_values(by=["ifScore"],ascending=False)
list3=scores.sort_values(by=["citations"],ascending=False)
list4=scores.sort_values(by=["citations"],ascending=False)
list6=scores.sort_values(by=["hIndex"],ascending=False)
list1[[ "Név","ifScore"]].head(10)


,Név,ifScore
65,Molontay Roland,49.187
20,Faragó István György,41.694
118,Vrana Péter,39.871
49,Kovács Mihály,38.434
38,Karátson János,35.507
86,Rónyai Lajos,33.444
5,Bárány Balázs,32.222
27,Gyurkovics Éva,27.860
119,Weiner Mihály,24.309
46,Koós Krisztiánné Szilágyi Brigitta,23.809


In [43]:

list3[[ "Név","citations"]].head(10)


,Név,citations
65,Molontay Roland,157
49,Kovács Mihály,148
5,Bárány Balázs,116
67,Mosonyi Milán,101
74,Nagy Marcell,97
119,Weiner Mihály,85
27,Gyurkovics Éva,80
20,Faragó István György,78
86,Rónyai Lajos,71
24,G. Horváth Ákos,70


In [44]:

list6[[ "Név","hIndex"]].head(10)

,Név,hIndex
65,Molontay Roland,8
5,Bárány Balázs,6
49,Kovács Mihály,6
74,Nagy Marcell,6
20,Faragó István György,5
119,Weiner Mihály,5
86,Rónyai Lajos,5
27,Gyurkovics Éva,4
24,G. Horváth Ákos,4
67,Mosonyi Milán,4


# with affiliationpeople

In [18]:
def load_pub_for_author(mtid):
    page = 1
    size = 1000
    tries = 1
    author_pubs = []
    while True:
        os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?cond=published;eq;true&cond=institutes;inia;10856&cond=core;eq;true&cond=authors.mtid;eq;{mtid}&sort=publishedYear,desc&sort=firstAuthor,asc&size={size}&fields=citations,pubStats&labelLang=hun&page={page}&format=json" >wget.out 2>wget.err')
        with open("tmp.json", "r") as f:
            try:
                dta = json.load(f)
                if page == 1 and len(dta["content"]) == 0:
                    return []
                author_pubs += dta["content"]
                if not dta["paging"]["last"]:
                    page += 1
                else:
                    break
            except:
                tries += 1
                print("Error, retrying")
                if tries > 30:
                    return []
    return author_pubs
for mtid in tqdm(staff_flt["MTMT"]):
    if not os.path.exists(f"3_3/{mtid}.json"):
        json.dump(load_pub_for_author(mtid), open(f"3_3/{mtid}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)

100%|█████████████████████████████████████| 119/119 [00:00<00:00, 114372.63it/s]


In [19]:
def score_person(mtid):
    if not os.path.exists(f'3_3/{mtid}.json'):
        raise Exception("Nincs letöltve a publikációs lista!")
    pubs = json.load(open(f'3_3/{mtid}.json'))
    score = score_publist([mtid], pubs)
    score["mtid"] = int(mtid)
    return score
    
def score_publist(mtid_list, pubs):
    score = {"pub_count": 0, "q_paper": 0, "q_n_paper": 0, "q_book": 0, "if": 0, "relif": 0, "ifcnt": 0, "if_norm": 0, "relif_norm": 0, "h": 0, "i": 0, "first_pub": 2100, "last_pub": 0}
    cit_list = []
    useShare = False
    ix = 0
    for pub in pubs:
        ix += 1
        if "error" in pub and pub["error"]!="VALIDATION_ERROR":
            continue
        if "category" not in pub or pub["category"]["label"]!="Tudományos":
            continue
        # compute share of the authos
        authors = 0
        share = len(mtid_list)/len(pub["authorships"]) if len(pub["authorships"])>0 else 0
        for a in pub["authorships"]:
            if "author" in a and a["author"]["mtid"] in mtid_list and a["authorTyped"]:
                authors += 1
        if authors < len(mtid_list):
            continue
        # compute number of pages
        plength = 0
        try:
            if "pageLength" in pub:
                plength = int(pub["pageLength"])
            elif "firstPage" in pub and "lastPage" in pub:
                plength = int(pub["lastPage"]) - int(pub["firstPage"]) + 1            
        except:
            pass
        # compute impact factor
        ifct, nrm = getif(pub)        
        # compute Q score
        if pub["otype"] == "JournalArticle" and "journal" in pub:
            if "fullPublication" in pub and pub["fullPublication"] and "reviewType" in pub["journal"] and pub["journal"]["reviewType"]=="REVIEWED" and "subType" in pub and \
                pub["subType"]["name"] in ["Szakcikk", "Összefoglaló cikk", "Konferenciaközlemény", "Rövid közlemény", "Sokszerzős vagy csoportos szerzőségű szakcikk"]:                
                if ifct > 0:
                    totalscr = max(0.6,ifct)
                elif "foreignEdition" in pub and pub["foreignEdition"]:
                    totalscr = 0.4
                else:
                    totalscr = 0.3
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share / nrm if nrm > 0 else totalscr * share
                if ifct > 0:
                    score["if"] += ifct
                    score["relif"] += totalscr * share
                    score["ifcnt"] += 1
                    score["if_norm"] += ifct / nrm if nrm > 0 else ifct
                    score["relif_norm"] += totalscr * share / nrm if nrm > 0 else totalscr * share
        elif "fullPublication" in pub and pub["fullPublication"] and \
            ((pub["type"]["label"]=="Könyvrészlet" and "subType" in pub and pub["subType"]["label"]=="Konferenciaközlemény (Könyvrészlet)") or pub["type"]["label"]=="Egyéb konferenciaközlemény"):                
            if plength >= 4:
                totalscr = 0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share
        elif pub["type"]["label"] == "Könyv" or \
            (pub["type"]["label"] == "Könyvrészlet" and "subType" in pub and pub["subType"]["label"] in ["Könyvfejezet (Könyvrészlet)", "Szaktanulmány (Könyvrészlet)"]):
            if plength >= 10:
                if plength >= 100:
                    totalscr = 2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 1
                else:
                    totalscr = (0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1) * math.floor(plength/10)
                score["q_book"] += totalscr * share
        # Process citations
        if "independentCitationCount" in pub:
            score["i"] += pub["independentCitationCount"]
            cit_list.append(pub["independentCitationCount"])
        if "publishedYear" in pub:
            score["last_pub"] = max(score["last_pub"], pub["publishedYear"])
            score["first_pub"] = min(score["first_pub"], pub["publishedYear"])
        score["pub_count"] += 1
    score["q"] = score["q_paper"] + np.minimum(score["q_book"], 3.0)
    score["qn"] = score["q_n_paper"] + np.minimum(score["q_book"], 3.0)
    # h-index
    cit_list.sort(reverse=True)
    while score["h"] < len(cit_list) and score["h"] < cit_list[score["h"]]:
        score["h"] += 1
        
    return score
def get_h_score(mtid):
    cit_list = []
    if os.path.exists(f'3_3/{mtid}.json'):
        with open(f'3_3/{mtid}.json') as f:
            try:
                js = json.load(f)
                for pub in js:
                    if "independentCitationCount" in pub:
                        cit_list.append(pub["independentCitationCount"])
            except:
                pass
    cit_list.sort(reverse=True)
    hindex = 0
    while hindex < len(cit_list) and hindex < cit_list[hindex]:
        hindex += 1
    return hindex

In [20]:
staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]
dta = {"MTMT": [], "pubCount": [], "qScore": [], "qnScore": [], "ifCount": [], "ifScore": [], "relifScore": [], "ifNormScore": [], "relifNormScore": [], "citations": [], "hIndex": [], "firstPub": [], "lastPub": []}
for idx, prs in staff_flt.iterrows():
    scr = score_person(prs["MTMT"])
    dta["MTMT"].append(scr["mtid"])
    dta["pubCount"].append(scr["pub_count"])
    dta["qScore"].append(scr["q"])
    dta["qnScore"].append(scr["qn"])
    dta["ifCount"].append(scr["ifcnt"])
    dta["ifScore"].append(scr["if"])
    dta["relifScore"].append(scr["relif"])
    dta["ifNormScore"].append(scr["if_norm"])
    dta["relifNormScore"].append(scr["relif_norm"])
    dta["citations"].append(scr["i"])
    dta["hIndex"].append(scr["h"])
    dta["firstPub"].append(scr["first_pub"])
    dta["lastPub"].append(scr["last_pub"])
node_person = pd.DataFrame(dta)
node_person

/tmp/ipykernel_271585/3273158239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]


,MTMT,pubCount,qScore,qnScore,ifCount,ifScore,relifScore,ifNormScore,relifNormScore,citations,hIndex,firstPub,lastPub
0,10012522,19,7.587500,5.739092,8,9.012,6.887500,6.284685,5.039092,76,5,1998,2020
1,10041405,9,1.950000,5.935340,3,0.490,1.200000,1.934906,5.185340,4,1,1982,2009
2,10011729,29,14.387283,10.581248,24,37.300,14.253950,27.004655,10.447914,216,8,2002,2021
3,10042381,13,1.633333,1.633333,0,0.000,0.000000,0.000000,0.000000,1,1,2008,2017
4,10011669,23,5.360267,2.617591,6,12.839,4.210267,4.332244,1.467591,49,4,1985,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,10019824,19,8.545417,6.720328,14,14.789,7.895417,11.317157,6.070328,179,5,2006,2022
115,10019824,19,8.545417,6.720328,14,14.789,7.895417,11.317157,6.070328,179,5,2006,2022
116,10042219,25,18.972517,9.137706,20,43.038,18.905850,19.959357,9.071039,44,4,2015,2023
117,10015273,25,16.528967,10.011277,24,39.862,16.395633,24.277772,9.877943,246,9,2002,2022


In [21]:
people=pd.read_csv('people_flt.csv')
scores=node_person
scores=pd.merge(scores, people[["MTMT", "Név", "Web"]], how='inner',  on=["MTMT"])
list1=scores.sort_values(by=["ifScore"],ascending=False)
list3=scores.sort_values(by=["citations"],ascending=False)
list4=scores.sort_values(by=["citations"],ascending=False)
list6=scores.sort_values(by=["hIndex"],ascending=False)
list1[[ "Név","ifScore"]].head(10)


,Név,ifScore
65,Molontay Roland,49.187
67,Mosonyi Milán,48.656
27,Gyurkovics Éva,48.258
118,Vrana Péter,43.038
5,Bárány Balázs,40.723
119,Weiner Mihály,39.862
20,Faragó István György,38.914
38,Karátson János,38.029
2,Bálint Péter,37.300
91,Simon Károly,35.464


In [22]:

list3[[ "Név","citations"]].head(10)


,Név,citations
67,Mosonyi Milán,830
47,Kornai András,553
27,Gyurkovics Éva,436
14,Csiszár Imre,434
91,Simon Károly,411
86,Rónyai Lajos,290
33,Horváth Miklós Tibor,266
59,Matolcsi Máté,265
84,Ráth Balázs,260
100,Szász Domokos,253


In [23]:

list6[[ "Név","hIndex"]].head(10)

,Név,hIndex
67,Mosonyi Milán,17
27,Gyurkovics Éva,12
91,Simon Károly,12
84,Ráth Balázs,10
24,G. Horváth Ákos,9
100,Szász Domokos,9
119,Weiner Mihály,9
47,Kornai András,9
5,Bárány Balázs,9
86,Rónyai Lajos,9


# last 5 years

In [24]:
def load_pub_for_author(mtid):
    page = 1
    size = 1000
    tries = 1
    author_pubs = []
    while True:
        os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?cond=published;eq;true&cond=institutes;inia;10856&cond=core;eq;true&cond=authors.mtid;eq;{mtid}&cond=publishedYear;range;2017%2C2023&sort=publishedYear,desc&sort=firstAuthor,asc&size={size}&fields=citations,pubStats&labelLang=hun&page={page}&format=json" >wget.out 2>wget.err')
        with open("tmp.json", "r") as f:
            try:
                dta = json.load(f)
                if page == 1 and len(dta["content"]) == 0:
                    return []
                author_pubs += dta["content"]
                if not dta["paging"]["last"]:
                    page += 1
                else:
                    break
            except:
                tries += 1
                print("Error, retrying")
                if tries > 30:
                    return []
    return author_pubs
for mtid in tqdm(staff_flt["MTMT"]):
    if not os.path.exists(f"3_4/{mtid}.json"):
        json.dump(load_pub_for_author(mtid), open(f"3_4/{mtid}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)

100%|█████████████████████████████████████| 119/119 [00:00<00:00, 148681.02it/s]


In [25]:
def score_person(mtid):
    if not os.path.exists(f'3_4/{mtid}.json'):
        raise Exception("Nincs letöltve a publikációs lista!")
    pubs = json.load(open(f'3_4/{mtid}.json'))
    score = score_publist([mtid], pubs)
    score["mtid"] = int(mtid)
    return score
    
def score_publist(mtid_list, pubs):
    score = {"pub_count": 0, "q_paper": 0, "q_n_paper": 0, "q_book": 0, "if": 0, "relif": 0, "ifcnt": 0, "if_norm": 0, "relif_norm": 0, "h": 0, "i": 0, "first_pub": 2100, "last_pub": 0}
    cit_list = []
    useShare = False
    ix = 0
    for pub in pubs:
        ix += 1
        if "error" in pub and pub["error"]!="VALIDATION_ERROR":
            continue
        if "category" not in pub or pub["category"]["label"]!="Tudományos":
            continue
        # compute share of the authos
        authors = 0
        share = len(mtid_list)/len(pub["authorships"]) if len(pub["authorships"])>0 else 0
        for a in pub["authorships"]:
            if "author" in a and a["author"]["mtid"] in mtid_list and a["authorTyped"]:
                authors += 1
        if authors < len(mtid_list):
            continue
        # compute number of pages
        plength = 0
        try:
            if "pageLength" in pub:
                plength = int(pub["pageLength"])
            elif "firstPage" in pub and "lastPage" in pub:
                plength = int(pub["lastPage"]) - int(pub["firstPage"]) + 1            
        except:
            pass
        # compute impact factor
        ifct, nrm = getif(pub)        
        # compute Q score
        if pub["otype"] == "JournalArticle" and "journal" in pub:
            if "fullPublication" in pub and pub["fullPublication"] and "reviewType" in pub["journal"] and pub["journal"]["reviewType"]=="REVIEWED" and "subType" in pub and \
                pub["subType"]["name"] in ["Szakcikk", "Összefoglaló cikk", "Konferenciaközlemény", "Rövid közlemény", "Sokszerzős vagy csoportos szerzőségű szakcikk"]:                
                if ifct > 0:
                    totalscr = max(0.6,ifct)
                elif "foreignEdition" in pub and pub["foreignEdition"]:
                    totalscr = 0.4
                else:
                    totalscr = 0.3
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share / nrm if nrm > 0 else totalscr * share
                if ifct > 0:
                    score["if"] += ifct
                    score["relif"] += totalscr * share
                    score["ifcnt"] += 1
                    score["if_norm"] += ifct / nrm if nrm > 0 else ifct
                    score["relif_norm"] += totalscr * share / nrm if nrm > 0 else totalscr * share
        elif "fullPublication" in pub and pub["fullPublication"] and \
            ((pub["type"]["label"]=="Könyvrészlet" and "subType" in pub and pub["subType"]["label"]=="Konferenciaközlemény (Könyvrészlet)") or pub["type"]["label"]=="Egyéb konferenciaközlemény"):                
            if plength >= 4:
                totalscr = 0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share
        elif pub["type"]["label"] == "Könyv" or \
            (pub["type"]["label"] == "Könyvrészlet" and "subType" in pub and pub["subType"]["label"] in ["Könyvfejezet (Könyvrészlet)", "Szaktanulmány (Könyvrészlet)"]):
            if plength >= 10:
                if plength >= 100:
                    totalscr = 2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 1
                else:
                    totalscr = (0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1) * math.floor(plength/10)
                score["q_book"] += totalscr * share
        # Process citations
        if "independentCitationCount" in pub:
            score["i"] += pub["independentCitationCount"]
            cit_list.append(pub["independentCitationCount"])
        if "publishedYear" in pub:
            score["last_pub"] = max(score["last_pub"], pub["publishedYear"])
            score["first_pub"] = min(score["first_pub"], pub["publishedYear"])
        score["pub_count"] += 1
    score["q"] = score["q_paper"] + np.minimum(score["q_book"], 3.0)
    score["qn"] = score["q_n_paper"] + np.minimum(score["q_book"], 3.0)
    # h-index
    cit_list.sort(reverse=True)
    while score["h"] < len(cit_list) and score["h"] < cit_list[score["h"]]:
        score["h"] += 1
        
    return score
def get_h_score(mtid):
    cit_list = []
    if os.path.exists(f'3_4/{mtid}.json'):
        with open(f'3_4/{mtid}.json') as f:
            try:
                js = json.load(f)
                for pub in js:
                    if "independentCitationCount" in pub:
                        cit_list.append(pub["independentCitationCount"])
            except:
                pass
    cit_list.sort(reverse=True)
    hindex = 0
    while hindex < len(cit_list) and hindex < cit_list[hindex]:
        hindex += 1
    return hindex

In [26]:
staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]
dta = {"MTMT": [], "pubCount": [], "qScore": [], "qnScore": [], "ifCount": [], "ifScore": [], "relifScore": [], "ifNormScore": [], "relifNormScore": [], "citations": [], "hIndex": [], "firstPub": [], "lastPub": []}
for idx, prs in staff_flt.iterrows():
    scr = score_person(prs["MTMT"])
    dta["MTMT"].append(scr["mtid"])
    dta["pubCount"].append(scr["pub_count"])
    dta["qScore"].append(scr["q"])
    dta["qnScore"].append(scr["qn"])
    dta["ifCount"].append(scr["ifcnt"])
    dta["ifScore"].append(scr["if"])
    dta["relifScore"].append(scr["relif"])
    dta["ifNormScore"].append(scr["if_norm"])
    dta["relifNormScore"].append(scr["relif_norm"])
    dta["citations"].append(scr["i"])
    dta["hIndex"].append(scr["h"])
    dta["firstPub"].append(scr["first_pub"])
    dta["lastPub"].append(scr["last_pub"])
node_person = pd.DataFrame(dta)
node_person

/tmp/ipykernel_271585/3273158239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]


,MTMT,pubCount,qScore,qnScore,ifCount,ifScore,relifScore,ifNormScore,relifNormScore,citations,hIndex,firstPub,lastPub
0,10012522,3,1.527500,0.762877,2,3.055,1.527500,1.525753,0.762877,17,2,2017,2020
1,10041405,0,0.000000,0.000000,0,0.000,0.000000,0.000000,0.000000,0,0,2100,0
2,10011729,8,3.312200,1.720991,6,10.801,3.178867,5.417736,1.587658,25,3,2017,2021
3,10042381,2,0.200000,0.200000,0,0.000,0.000000,0.000000,0.000000,0,0,2017,2017
4,10011669,5,1.935000,0.879538,1,6.140,1.535000,1.918151,0.479538,8,1,2017,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,10019824,7,4.052500,2.776344,6,6.675,4.052500,4.616211,2.776344,11,3,2017,2022
115,10019824,7,4.052500,2.776344,6,6.675,4.052500,4.616211,2.776344,11,3,2017,2022
116,10042219,23,17.711183,8.405214,18,39.871,17.644517,18.081074,8.338547,30,3,2017,2023
117,10015273,13,10.910750,5.790201,13,24.309,10.910750,13.781714,5.790201,85,5,2017,2022


In [27]:
people=pd.read_csv('people_flt.csv')
scores=node_person
scores=pd.merge(scores, people[["MTMT", "Név", "Web"]], how='inner',  on=["MTMT"])
list1=scores.sort_values(by=["ifScore"],ascending=False)
list3=scores.sort_values(by=["citations"],ascending=False)

list6=scores.sort_values(by=["hIndex"],ascending=False)
list1[[ "Név","ifScore"]].head(10)


,Név,ifScore
65,Molontay Roland,49.187
118,Vrana Péter,39.871
38,Karátson János,32.513
5,Bárány Balázs,30.692
20,Faragó István György,29.436
27,Gyurkovics Éva,25.098
119,Weiner Mihály,24.309
74,Nagy Marcell,22.430
110,Tóth János,19.989
46,Koós Krisztiánné Szilágyi Brigitta,19.724


In [28]:

list3[[ "Név","citations"]].head(10)


,Név,citations
65,Molontay Roland,157
5,Bárány Balázs,115
67,Mosonyi Milán,101
74,Nagy Marcell,97
119,Weiner Mihály,85
27,Gyurkovics Éva,71
24,G. Horváth Ákos,69
86,Rónyai Lajos,66
41,Kiss Krisztina,63
43,Kiss Sándor,45


In [29]:

list6[[ "Név","hIndex"]].head(10)

,Név,hIndex
65,Molontay Roland,8
5,Bárány Balázs,6
74,Nagy Marcell,6
119,Weiner Mihály,5
86,Rónyai Lajos,5
67,Mosonyi Milán,4
27,Gyurkovics Éva,4
24,G. Horváth Ákos,4
114,Vető Bálint,3
110,Tóth János,3


# regardless affiliation

In [30]:

def score_person(mtid):
    if not os.path.exists(f'3_1/{mtid}.json'):
        raise Exception("Nincs letöltve a publikációs lista!")
    pubs = json.load(open(f'3_1/{mtid}.json'))
    score = score_publist([mtid], pubs)
    score["mtid"] = int(mtid)
    return score
    
def score_publist(mtid_list, pubs):
    score = {"pub_count": 0, "q_paper": 0, "q_n_paper": 0, "q_book": 0, "if": 0, "relif": 0, "ifcnt": 0, "if_norm": 0, "relif_norm": 0, "h": 0, "i": 0, "first_pub": 2100, "last_pub": 0}
    cit_list = []
    useShare = False
    ix = 0
    for pub in pubs:
        ix += 1
        if "error" in pub and pub["error"]!="VALIDATION_ERROR":
            continue
        if "category" not in pub or pub["category"]["label"]!="Tudományos":
            continue
        # compute share of the authos
        authors = 0
        share = len(mtid_list)/len(pub["authorships"]) if len(pub["authorships"])>0 else 0
        for a in pub["authorships"]:
            if "author" in a and a["author"]["mtid"] in mtid_list and a["authorTyped"]:
                authors += 1
        if authors < len(mtid_list):
            continue
        # compute number of pages
        plength = 0
        try:
            if "pageLength" in pub:
                plength = int(pub["pageLength"])
            elif "firstPage" in pub and "lastPage" in pub:
                plength = int(pub["lastPage"]) - int(pub["firstPage"]) + 1            
        except:
            pass
        # compute impact factor
        ifct, nrm = getif(pub)        
        # compute Q score
        if pub["otype"] == "JournalArticle" and "journal" in pub:
            if "fullPublication" in pub and pub["fullPublication"] and "reviewType" in pub["journal"] and pub["journal"]["reviewType"]=="REVIEWED" and "subType" in pub and \
                pub["subType"]["name"] in ["Szakcikk", "Összefoglaló cikk", "Konferenciaközlemény", "Rövid közlemény", "Sokszerzős vagy csoportos szerzőségű szakcikk"]:                
                if ifct > 0:
                    totalscr = max(0.6,ifct)
                elif "foreignEdition" in pub and pub["foreignEdition"]:
                    totalscr = 0.4
                else:
                    totalscr = 0.3
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share / nrm if nrm > 0 else totalscr * share
                if ifct > 0:
                    score["if"] += ifct
                    score["relif"] += totalscr * share
                    score["ifcnt"] += 1
                    score["if_norm"] += ifct / nrm if nrm > 0 else ifct
                    score["relif_norm"] += totalscr * share / nrm if nrm > 0 else totalscr * share
        elif "fullPublication" in pub and pub["fullPublication"] and \
            ((pub["type"]["label"]=="Könyvrészlet" and "subType" in pub and pub["subType"]["label"]=="Konferenciaközlemény (Könyvrészlet)") or pub["type"]["label"]=="Egyéb konferenciaközlemény"):                
            if plength >= 4:
                totalscr = 0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share
        elif pub["type"]["label"] == "Könyv" or \
            (pub["type"]["label"] == "Könyvrészlet" and "subType" in pub and pub["subType"]["label"] in ["Könyvfejezet (Könyvrészlet)", "Szaktanulmány (Könyvrészlet)"]):
            if plength >= 10:
                if plength >= 100:
                    totalscr = 2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 1
                else:
                    totalscr = (0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1) * math.floor(plength/10)
                score["q_book"] += totalscr * share
        # Process citations
        independentCitation=0
        if "pubStats" in pub:
            for a in pub["pubStats"]["years"]:
                if a["year"]>2016:
                    independentCitation+=a["independentCitationCount"]
        if "independentCitationCount" in pub:
            score["i"] += independentCitation
            cit_list.append(independentCitation)

        if "publishedYear" in pub:
            score["last_pub"] = max(score["last_pub"], pub["publishedYear"])
            score["first_pub"] = min(score["first_pub"], pub["publishedYear"])
        score["pub_count"] += 1
    score["q"] = score["q_paper"] + np.minimum(score["q_book"], 3.0)
    score["qn"] = score["q_n_paper"] + np.minimum(score["q_book"], 3.0)
    # h-index
    cit_list.sort(reverse=True)
    while score["h"] < len(cit_list) and score["h"] < cit_list[score["h"]]:
        score["h"] += 1
        
    return score
def get_h_score(mtid):
    cit_list = []
    if os.path.exists(f'3_1/{mtid}.json'):
        with open(f'3_1/{mtid}.json') as f:
            try:
                js = json.load(f)
                for pub in js:
                    independentCitation=0
                    if "pubStats" in pub:
                        for a in pub["pubStats"]["years"]:
                            if a["year"]>2016:
                                independentCitation+=a["independentCitationCount"]
                    if "independentCitationCount" in pub:
                        cit_list.append(independentCitation)
            except:
                pass
    cit_list.sort(reverse=True)
    hindex = 0
    while hindex < len(cit_list) and hindex < cit_list[hindex]:
        hindex += 1
    return hindex

In [31]:
staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]
dta = {"MTMT": [], "pubCount": [], "qScore": [], "qnScore": [], "ifCount": [], "ifScore": [], "relifScore": [], "ifNormScore": [], "relifNormScore": [], "citations": [], "hIndex": [], "firstPub": [], "lastPub": []}
for idx, prs in staff_flt.iterrows():
    scr = score_person(prs["MTMT"])
    dta["MTMT"].append(scr["mtid"])
    dta["pubCount"].append(scr["pub_count"])
    dta["qScore"].append(scr["q"])
    dta["qnScore"].append(scr["qn"])
    dta["ifCount"].append(scr["ifcnt"])
    dta["ifScore"].append(scr["if"])
    dta["relifScore"].append(scr["relif"])
    dta["ifNormScore"].append(scr["if_norm"])
    dta["relifNormScore"].append(scr["relif_norm"])
    dta["citations"].append(scr["i"])
    dta["hIndex"].append(scr["h"])
    dta["firstPub"].append(scr["first_pub"])
    dta["lastPub"].append(scr["last_pub"])
node_person = pd.DataFrame(dta)
node_person

/tmp/ipykernel_271585/3273158239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]


,MTMT,pubCount,qScore,qnScore,ifCount,ifScore,relifScore,ifNormScore,relifNormScore,citations,hIndex,firstPub,lastPub
0,10012522,23,9.162250,6.800885,10,10.856,8.162250,7.796477,5.800885,34,3,1998,2020
1,10041405,36,7.200000,16.093861,6,0.918,2.400000,4.281639,11.293861,1,1,1970,2009
2,10011729,33,15.860783,12.435711,26,39.425,15.727450,28.957170,12.302377,87,5,1997,2021
3,10042381,15,1.733333,1.733333,0,0.000,0.000000,0.000000,0.000000,0,0,2008,2017
4,10011669,31,7.139625,3.533433,10,21.269,5.916898,8.404271,2.310706,71,6,1985,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,10019824,20,8.937917,7.132186,15,15.574,8.287917,12.140872,6.482186,118,4,2006,2022
115,10019824,20,8.937917,7.132186,15,15.574,8.287917,12.140872,6.482186,118,4,2006,2022
116,10042219,39,29.287500,14.404563,30,67.256,29.220833,31.425362,14.337897,64,5,2008,2023
117,10015273,33,25.605867,15.803946,30,53.344,25.372533,33.254772,15.570612,200,8,2001,2022


In [32]:
people=pd.read_csv('people_flt.csv')
scores=node_person
scores=pd.merge(scores, people[["MTMT", "Név", "Web"]], how='inner',  on=["MTMT"])
list1=scores.sort_values(by=["ifScore"],ascending=False)
list3=scores.sort_values(by=["citations"],ascending=False)
list4=scores.sort_values(by=["citations"],ascending=False)
list6=scores.sort_values(by=["hIndex"],ascending=False)
list1[[ "Név","ifScore"]].head(10)

,Név,ifScore
63,Molnár Lajos Gábor,100.458
20,Faragó István György,93.602
38,Karátson János,75.252
86,Rónyai Lajos,70.879
110,Tóth János,69.834
118,Vrana Péter,67.256
49,Kovács Mihály,63.502
59,Matolcsi Máté,56.742
119,Weiner Mihály,53.344
18,Etesi Gábor,53.291


In [33]:
list3[[ "Név","citations"]].head(10)

,Név,citations
14,Csiszár Imre,3246
47,Kornai András,1087
63,Molnár Lajos Gábor,693
67,Mosonyi Milán,529
49,Kovács Mihály,506
86,Rónyai Lajos,452
59,Matolcsi Máté,447
20,Faragó István György,341
27,Gyurkovics Éva,257
84,Ráth Balázs,216


In [34]:
list6[[ "Név","hIndex"]].head(10)

,Név,hIndex
14,Csiszár Imre,24
49,Kovács Mihály,16
47,Kornai András,16
63,Molnár Lajos Gábor,12
67,Mosonyi Milán,12
91,Simon Károly,9
5,Bárány Balázs,9
86,Rónyai Lajos,9
84,Ráth Balázs,9
19,Fábián Csaba,8


In [35]:
def score_person(mtid):
    if not os.path.exists(f'3_3/{mtid}.json'):
        raise Exception("Nincs letöltve a publikációs lista!")
    pubs = json.load(open(f'3_3/{mtid}.json'))
    score = score_publist([mtid], pubs)
    score["mtid"] = int(mtid)
    return score
    
def score_publist(mtid_list, pubs):
    score = {"pub_count": 0, "q_paper": 0, "q_n_paper": 0, "q_book": 0, "if": 0, "relif": 0, "ifcnt": 0, "if_norm": 0, "relif_norm": 0, "h": 0, "i": 0, "first_pub": 2100, "last_pub": 0}
    cit_list = []
    useShare = False
    ix = 0
    for pub in pubs:
        ix += 1
        if "error" in pub and pub["error"]!="VALIDATION_ERROR":
            continue
        if "category" not in pub or pub["category"]["label"]!="Tudományos":
            continue
        # compute share of the authos
        authors = 0
        share = len(mtid_list)/len(pub["authorships"]) if len(pub["authorships"])>0 else 0
        for a in pub["authorships"]:
            if "author" in a and a["author"]["mtid"] in mtid_list and a["authorTyped"]:
                authors += 1
        if authors < len(mtid_list):
            continue
        # compute number of pages
        plength = 0
        try:
            if "pageLength" in pub:
                plength = int(pub["pageLength"])
            elif "firstPage" in pub and "lastPage" in pub:
                plength = int(pub["lastPage"]) - int(pub["firstPage"]) + 1            
        except:
            pass
        # compute impact factor
        ifct, nrm = getif(pub)        
        # compute Q score
        if pub["otype"] == "JournalArticle" and "journal" in pub:
            if "fullPublication" in pub and pub["fullPublication"] and "reviewType" in pub["journal"] and pub["journal"]["reviewType"]=="REVIEWED" and "subType" in pub and \
                pub["subType"]["name"] in ["Szakcikk", "Összefoglaló cikk", "Konferenciaközlemény", "Rövid közlemény", "Sokszerzős vagy csoportos szerzőségű szakcikk"]:                
                if ifct > 0:
                    totalscr = max(0.6,ifct)
                elif "foreignEdition" in pub and pub["foreignEdition"]:
                    totalscr = 0.4
                else:
                    totalscr = 0.3
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share / nrm if nrm > 0 else totalscr * share
                if ifct > 0:
                    score["if"] += ifct
                    score["relif"] += totalscr * share
                    score["ifcnt"] += 1
                    score["if_norm"] += ifct / nrm if nrm > 0 else ifct
                    score["relif_norm"] += totalscr * share / nrm if nrm > 0 else totalscr * share
        elif "fullPublication" in pub and pub["fullPublication"] and \
            ((pub["type"]["label"]=="Könyvrészlet" and "subType" in pub and pub["subType"]["label"]=="Konferenciaközlemény (Könyvrészlet)") or pub["type"]["label"]=="Egyéb konferenciaközlemény"):                
            if plength >= 4:
                totalscr = 0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share
        elif pub["type"]["label"] == "Könyv" or \
            (pub["type"]["label"] == "Könyvrészlet" and "subType" in pub and pub["subType"]["label"] in ["Könyvfejezet (Könyvrészlet)", "Szaktanulmány (Könyvrészlet)"]):
            if plength >= 10:
                if plength >= 100:
                    totalscr = 2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 1
                else:
                    totalscr = (0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1) * math.floor(plength/10)
                score["q_book"] += totalscr * share
        # Process citations
        independentCitation=0
        if "pubStats" in pub:
            for a in pub["pubStats"]["years"]:
                if a["year"]>2016:
                    independentCitation+=a["independentCitationCount"]
        if "independentCitationCount" in pub:
            score["i"] += independentCitation
            cit_list.append(independentCitation)

        if "publishedYear" in pub:
            score["last_pub"] = max(score["last_pub"], pub["publishedYear"])
            score["first_pub"] = min(score["first_pub"], pub["publishedYear"])
        score["pub_count"] += 1
    score["q"] = score["q_paper"] + np.minimum(score["q_book"], 3.0)
    score["qn"] = score["q_n_paper"] + np.minimum(score["q_book"], 3.0)
    # h-index
    cit_list.sort(reverse=True)
    while score["h"] < len(cit_list) and score["h"] < cit_list[score["h"]]:
        score["h"] += 1
        
    return score
def get_h_score(mtid):
    cit_list = []
    if os.path.exists(f'3_3/{mtid}.json'):
        with open(f'3_3/{mtid}.json') as f:
            try:
                js = json.load(f)
                for pub in js:
                    independentCitation=0
                    if "pubStats" in pub:
                        for a in pub["pubStats"]["years"]:
                            if a["year"]>2016:
                                independentCitation+=a["independentCitationCount"]
                    if "independentCitationCount" in pub:
                        cit_list.append(independentCitation)
            except:
                pass
    cit_list.sort(reverse=True)
    hindex = 0
    while hindex < len(cit_list) and hindex < cit_list[hindex]:
        hindex += 1
    return hindex

In [36]:
staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]
dta = {"MTMT": [], "pubCount": [], "qScore": [], "qnScore": [], "ifCount": [], "ifScore": [], "relifScore": [], "ifNormScore": [], "relifNormScore": [], "citations": [], "hIndex": [], "firstPub": [], "lastPub": []}
for idx, prs in staff_flt.iterrows():
    scr = score_person(prs["MTMT"])
    dta["MTMT"].append(scr["mtid"])
    dta["pubCount"].append(scr["pub_count"])
    dta["qScore"].append(scr["q"])
    dta["qnScore"].append(scr["qn"])
    dta["ifCount"].append(scr["ifcnt"])
    dta["ifScore"].append(scr["if"])
    dta["relifScore"].append(scr["relif"])
    dta["ifNormScore"].append(scr["if_norm"])
    dta["relifNormScore"].append(scr["relif_norm"])
    dta["citations"].append(scr["i"])
    dta["hIndex"].append(scr["h"])
    dta["firstPub"].append(scr["first_pub"])
    dta["lastPub"].append(scr["last_pub"])
node_person = pd.DataFrame(dta)
node_person

/tmp/ipykernel_271585/3273158239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]


,MTMT,pubCount,qScore,qnScore,ifCount,ifScore,relifScore,ifNormScore,relifNormScore,citations,hIndex,firstPub,lastPub
0,10012522,19,7.587500,5.739092,8,9.012,6.887500,6.284685,5.039092,32,3,1998,2020
1,10041405,9,1.950000,5.935340,3,0.490,1.200000,1.934906,5.185340,0,0,1982,2009
2,10011729,29,14.387283,10.581248,24,37.300,14.253950,27.004655,10.447914,87,5,2002,2021
3,10042381,13,1.633333,1.633333,0,0.000,0.000000,0.000000,0.000000,0,0,2008,2017
4,10011669,23,5.360267,2.617591,6,12.839,4.210267,4.332244,1.467591,32,3,1985,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,10019824,19,8.545417,6.720328,14,14.789,7.895417,11.317157,6.070328,111,4,2006,2022
115,10019824,19,8.545417,6.720328,14,14.789,7.895417,11.317157,6.070328,111,4,2006,2022
116,10042219,25,18.972517,9.137706,20,43.038,18.905850,19.959357,9.071039,40,3,2015,2023
117,10015273,25,16.528967,10.011277,24,39.862,16.395633,24.277772,9.877943,159,7,2002,2022


In [37]:
people=pd.read_csv('people_flt.csv')
scores=node_person
scores=pd.merge(scores, people[["MTMT", "Név", "Web"]], how='inner',  on=["MTMT"])
list1=scores.sort_values(by=["ifScore"],ascending=False)
list3=scores.sort_values(by=["citations"],ascending=False)
list4=scores.sort_values(by=["citations"],ascending=False)
list6=scores.sort_values(by=["hIndex"],ascending=False)
list1[[ "Név","ifScore"]].head(10)

,Név,ifScore
65,Molontay Roland,49.187
67,Mosonyi Milán,48.656
27,Gyurkovics Éva,48.258
118,Vrana Péter,43.038
5,Bárány Balázs,40.723
119,Weiner Mihály,39.862
20,Faragó István György,38.914
38,Karátson János,38.029
2,Bálint Péter,37.300
91,Simon Károly,35.464


In [38]:
list3[[ "Név","citations"]].head(10)

,Név,citations
67,Mosonyi Milán,529
47,Kornai András,271
27,Gyurkovics Éva,248
5,Bárány Balázs,197
86,Rónyai Lajos,186
91,Simon Károly,173
65,Molontay Roland,161
84,Ráth Balázs,160
119,Weiner Mihály,159
14,Csiszár Imre,154


In [39]:
list6[[ "Név","hIndex"]].head(10)

,Név,hIndex
67,Mosonyi Milán,12
5,Bárány Balázs,9
65,Molontay Roland,8
91,Simon Károly,8
27,Gyurkovics Éva,8
84,Ráth Balázs,8
86,Rónyai Lajos,7
24,G. Horváth Ákos,7
47,Kornai András,7
119,Weiner Mihály,7
